In [1]:
import pandas as pd

In [5]:
LAGT = pd.read_json("../data/large_files/LAGT_paul.json")

In [6]:
christian_authors = LAGT[LAGT["provenience"]=="christian"]["author"].unique().tolist()
print(christian_authors)

['Gospel of Matthew', 'Gospel of Mark', 'Luke-Acts', 'Johnannine literature (New Testament)', 'Paul of Tarsus', 'Pauline literature', 'Hebrews (New Testament)', 'James (New Testament)', '1 Peter (New Testament)', '2 Peter (New Testament)', 'Jude (New Testament)', 'Revelation (New Testament)', 'Acta Joannis (recensio)', 'Clement of Alexandria', 'Maximus of Tyre', 'Justin Martyr', 'Pseudo-Justin Martyr', 'Athenagoras', 'Barnabas', 'Clement of Rome', 'Didache', 'Dionysius of Corinth', 'The Shepherd of Hermas', 'Ignatius of Antioch', 'Irenaeus', 'Martyrdom of Polycarp', 'Polycarp', 'Seniores Apud Irenaeum', 'Theophilus of Antioch', 'Tatianus', 'The Passion of Saints Perpetua and Felicity', 'Eusebius', 'Eusebius of Caesarea', 'Epiphanius', 'Gregorius Nazianzenus', 'pseudo-Menander', 'Pappus Alexandrinus', 'Athanasius of Alexandria', 'Basil of Caesarea', 'Origen', 'Sozomen', 'Socrates of Constantinople', 'Philostorgius', 'Hippolytus of Rome', 'Mark the Deacon', 'Marcianus of Heraclea', 'Mari